Capítulo 6

6.3 Patrones de organización de datos.

6.3.1 Estructuración jerárquica

Prototipo - Este ejemplo de patrón de organización de datos consiste en que dada una estructura de datos en formato tabla, ésta es convertida a otro tipo de estructura en un formato jerarquizado XML.

In [32]:
%%writefile jerarquico.py
#!/usr/bin/env python
#Usamos el archivo foros.csv

from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol
import xmlify

class jerarquico(MRJob):
    
    OUTPUT_PROTOCOL = RawValueProtocol
    
    def mapper(self,_, line):
        linea=line.split(";") 
        
        mensaje=linea[4] # Recogemos el mensaje de la posición 4 de la línea
        tipoMensaje=linea[5] #Recogemos de la posición 5, si es una pregunta o respuesta
       
        if tipoMensaje=="question": 
            idMensaje=linea[0] #Almacenamos el id único del mensaje
            yield idMensaje,(tipoMensaje,mensaje)
        else:
            idMensaje=linea[7] #Almacenamos el identificador del mensaje idMensaje  
            yield idMensaje,(tipoMensaje,mensaje)
     
    def reducer(self, key, values):
        listaValores=[]
        listaPrincipal=[]
        listaAuxiliar=[] 
        
        for v in values: #Metemos los valores que vienen en un matriz
            listaValores.append(v) #Matriz que contiene el tipo de mensaje y el mensaje asociado 
        
        for valor in listaValores:
            if valor[0]=="question":#Si es una pregunta la metemos en la lista principal
                listaPrincipal.append(valor[1])
            else:
                listaAuxiliar.append(valor[1]) # Si son respuestas, las vamos agregando a una lista
        
        listaPrincipal.append(listaAuxiliar) #agregamos la lista de respuestas a la lista principal
        
        #Conversion a XML indicando en el raiz el id del mensaje
        yield "Creada linea XML: " ,xmlify.dumps(listaPrincipal,root = key) 
        
if __name__ == '__main__':
    jerarquico.run()

Overwriting jerarquico.py


In [31]:
!python jerarquico.py archivos_datos/foros.csv > jerarquico.xml

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\MRSANC~1\AppData\Local\Temp\jerarquico.mrsanchez.20201130.201929.316714
Running step 1 of 1...
job output is in C:\Users\MRSANC~1\AppData\Local\Temp\jerarquico.mrsanchez.20201130.201929.316714\output
Streaming final output from C:\Users\MRSANC~1\AppData\Local\Temp\jerarquico.mrsanchez.20201130.201929.316714\output...
Removing temp directory C:\Users\MRSANC~1\AppData\Local\Temp\jerarquico.mrsanchez.20201130.201929.316714...


Capítulo 6

6.3 Patrones de organización de datos.

6.3.1 Estructuración jerárquica

Prototipo - Esta segunda versión vamos a usar la estructura diccionarios de Python

In [26]:
%%writefile jerarquicoV2.py
#!/usr/bin/env python
#Usamos el archivo foros.csv

from mrjob.job import MRJob
from mrjob.protocol import RawValueProtocol
import xmlify

class jerarquicoV2(MRJob):
    OUTPUT_PROTOCOL = RawValueProtocol
    def mapper(self,_, line):
        linea=line.split(";") # Cada línea es un mensaje del foro (pregunta, respuesta o comentario)
        
        mensaje=linea[4] # Recogemos el mensaje de la posición 4 de la línea
        tipoMensaje=linea[5] #Recogemos de la posición 5, si es una pregunta, respuesta o comentario
        
        if tipoMensaje=="question": 
            idMensaje=linea[0] #Almacenamos el id único del mensaje
            yield idMensaje,(tipoMensaje,mensaje)
        else:
            idPadre=linea[7] #Almacenamos el identificador del mensaje idMensaje  
            yield idPadre,(tipoMensaje,mensaje)
     
    def reducer(self, key, values):
        diccionario=dict() #Para el caso que usemos diccionarios
        matrizParaXML=[]
        listaPrincipal=[]
        listaAuxiliar=[]
        
        for v in values: #Metemos los valores que vienen en un matriz
            matrizParaXML.append(v) #Matriz que contiene el tipo de mensaje y el mensaje asociado 
        
        for valor in matrizParaXML:
            if valor[0]=="question":#Si es una pregunta la metemos en la lista principal
                listaPrincipal.append(valor[1])
            else:
                listaAuxiliar.append(valor[1]) # Si son respuestas, las vamos agregando a una lista
        
        listaPrincipal.append(listaAuxiliar) #agregamos la lista de respuestas a la lista principal 
        diccionario[key]=listaPrincipal #Para el caso que usemos diccionarios
        yield key,xmlify.dumps(diccionario) # Conversion a XML para el caso que usemos diccionarios
        
if __name__ == '__main__':
    jerarquicoV2.run()

Overwriting jerarquicoV2.py


In [27]:
!python jerarquicoV2.py archivos_datos/foros.csv > jerarquicoV2.xml

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\MRSANC~1\AppData\Local\Temp\jerarquicoV2.mrsanchez.20201130.201330.069489
Running step 1 of 1...
job output is in C:\Users\MRSANC~1\AppData\Local\Temp\jerarquicoV2.mrsanchez.20201130.201330.069489\output
Streaming final output from C:\Users\MRSANC~1\AppData\Local\Temp\jerarquicoV2.mrsanchez.20201130.201330.069489\output...
Removing temp directory C:\Users\MRSANC~1\AppData\Local\Temp\jerarquicoV2.mrsanchez.20201130.201330.069489...
